## Stream Gauge Data on the Ohio River at Cannelton Dam at Cannelton, IN

In [1]:
#Import packages

import subprocess
from io import BytesIO

import folium
import hvplot.pandas
import pandas as pd
import requests

In [2]:
# Locate your Stream Gauge on a map
# 
sg_lat = 37.89950088
sg_lon = -86.70554179

# Initialize map and tweak settings
m = folium.Map(
    # Location to display
    location=(sg_lat, sg_lon),
    # Turns off annoying zooming while trying to scroll to the next cell
    scrollWheelZoom=False)

# Put a marker at the stream gauge location
folium.Marker([sg_lat, sg_lon], 
              popup="Stream Gauge on the Ohio River at Cannelton Dam at"
               "Cannelton, IN"
              ).add_to(m)

# Display the map
m

In [3]:
# Pull data from National Water Information System using API request

nwis_url = (
    "https://waterdata.usgs.gov/nwis/dv"
    "?cb_00060=on"
    "&format=rdb"
    "&site_no=03303280"
    "&legacy="
    "&referred_module=sw"
    "&period="
    "&begin_date=1975-10-19"
    "&end_date=2023-09-20"
)

# Send an HTTP GET request to the URL
nwis_response = requests.get(nwis_url)
nwis_response.raise_for_status()



In [4]:
# Print the top of the data
# for i, line in enumerate(nwis_response.content.splitlines()[:10]):
#    print(i, line)

In [5]:
# Take a look at the data. What got downloaded?
# for i, line in enumerate(nwis_response.content.splitlines()[:35]):
#    # Skip commented lines
#    if not line.startswith(b'#'):
#        print(i, line)

In [6]:
# Parse data and label columns

ohio_q_df = pd.read_csv(
    BytesIO(nwis_response.content),
    comment="#",
    delimiter="\t",
    skiprows=[29,30],
    names=["agency_cd", "site_no", "datetime", "streamflow_cfs", "code"],
    index_col="datetime",
    parse_dates=True,
 #   na_values='P'
)



In [7]:
# Parse data during Largest Flood of the 21st Century

ohio_flood_df = ohio_q_df['2017-10':'2019-09']


# The Ohio River Stream Flowrates vary seasonally

# The highest flowrates typically occur between February and March 
# with flowrates varying between 350,000 cfs and 550,000 cfs.

# The lowest flowrates occur between June and December 
# and are between 25,000 and 30,000 cfs.

In [8]:
# Plot the Shorter Time Span

ohio_q_df.hvplot(y='streamflow_cfs',
                     title='Ohio River Mean Daily Flowrates from Oct 1975 to Present',
                     xlabel='Time (month/year)',
                     ylabel='Flowrate (in cfs)')

/opt/conda/lib/python3.10/site-packages/holoviews/core/data/pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type
/opt/conda/lib/python3.10/site-packages/holoviews/core/data/pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type


:Curve   [datetime]   (streamflow_cfs)

# Two great flood events have occurred since the Cannelton Dam stream 
# gauge has been active. 
# March 8th, 1997 had a flowrate of 735,000 cfs
# February 27, 2018 had a flowrate 761,000 cfs.


In [9]:
# PLot all data for the Cannelton Dam on the Ohio River
ohio_flood_df.hvplot(y='streamflow_cfs',
                     title='Ohio River Mean Daily Flowrates from Oct 2017 to Sept 2019',
                     xlabel='Time (month/year)',
                     ylabel='Flowrate (in cfs)')


/opt/conda/lib/python3.10/site-packages/holoviews/core/data/pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type
/opt/conda/lib/python3.10/site-packages/holoviews/core/data/pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type


:Curve   [datetime]   (streamflow_cfs)

In [10]:
# Resample the data set using Annual Averages of all the Cannelton Dam Data

ohio_ann_max_q_df = ohio_q_df[['streamflow_cfs']].resample('AS').max()


# The flowrates of the great flood events were so significant that they
# also increased the annual flowrate averaged over a year. Notice the
# peaks in the diagram below. 

In [11]:
# Plot the Mean Annual Flowrates averaged over the year from 1975 to Present

ohio_ann_max_q_df.hvplot(y='streamflow_cfs',
                     title='Ohio River Mean Annual Flowrates from Oct 1975 to Present',
                     xlabel='Time (month/year)',
                     ylabel='Flowrate (in cfs)')

/opt/conda/lib/python3.10/site-packages/holoviews/core/data/pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type
/opt/conda/lib/python3.10/site-packages/holoviews/core/data/pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type


:Curve   [datetime]   (streamflow_cfs)

In [12]:
%%capture
%%bash
jupyter nbconvert ohio_flood_data.ipynb --to html --no-input